### 1 - Import libs

**ONLY EXECUTE THIS CELL**

In [1]:
import time 
from pathlib import Path
from datetime import datetime as dt

import ee
from sepal_ui.mapping import SepalMap
import geemap
# initialize EE    
try:
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')
except:
    ee.Authenticate()
    ee.Initialize(opt_url='https://earthengine-highvolume.googleapis.com')

import helpers as h

installed = !pip freeze | grep "retry=="
if not installed:
    !pip install --user retry

Styles()

/usr/local/lib/python3.8/dist-packages/bfast/__init__.py:10: UserWarning: cupy is not available in this environment, GPU fonctionnalities won't be available
  warn("cupy is not available in this environment, GPU fonctionnalities won't be available")


### 2 - Basic Input Variables

**FILL IN YOUR INPUTS**

In [2]:
# Area of Interest
import ee
import geemap

# country case example
#country = 'Kenya'   
#aoi = ee.FeatureCollection("FAO/GAUL/2015/level1").filter(ee.Filter.eq('ADM0_NAME', country)) # here any feature collection can be selected

#aoi = ee.FeatureCollection("projects/ee-erith72022/assets/experimento_ecorregiones_CO_12")
aoi = ee.FeatureCollection("projects/ee-erith72022/assets/experimento_ecorregiones_AZ_03")
# Point Feature Collection from where to extract points
#fc = ee.FeatureCollection("my_featurecollection_path").filterBounds(aoi)
#point_id_name = "PLOTID"  # the column/property within your point feature collection from which to take the UNIQUE point id

#fc = ee.FeatureCollection("users/impressfao/centre_2k_grid").filterBounds(aoi)
#fc = ee.FeatureCollection("projects/ee-erith72022/assets/coe12").filterBounds(aoi)
fc = ee.FeatureCollection("projects/ee-erith72022/assets/AZ03").filterBounds(aoi)
#point_id_name = "bb_tileID"
#point_id_name = "Index"
point_id_name = "plotid"
print(fc.size().getInfo())
print(fc.getInfo())

grid_size = 0.1  # that's the size of the grid we are parallelizing on
workers = 12 # number of parallel EE requests

# Time of interest (onsidering the historical period)
start_date = "2012-01-01"
end_date = "2021-12-31"

# Monitoring period
start_monitor = "2016-01-01"

# Directory where output and temp files will go
outdir = 'resultsMAATE_az03'

# Select algorithms to run
cusum_deforest = True
bfast_monitor = True
bs_slope = True
ts_metrics = True
ccdc = False
global_products = True

band='ndvi' # other choices: ndfi, ndmi, mndwi

### DO NOT CHANGE YET ###
satellite='Landsat'

204
{'type': 'FeatureCollection', 'columns': {'CC20132014': 'String', 'CC20152016': 'String', 'CC20172018': 'String', 'CC2019': 'String', 'ESTRATOS_P': 'String', 'RASTERVALU': 'Long', 'ecorregion': 'String', 'plotid': 'Float', 'system:index': 'String'}, 'version': 1656951333294856, 'id': 'projects/ee-erith72022/assets/AZ03', 'properties': {'system:asset_size': 17275}, 'features': [{'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-76.649891463149, -2.367969941624006]}, 'id': '00000000000000000038', 'properties': {'CC20132014': 'NBE', 'CC20152016': 'NBE', 'CC20172018': 'NBE', 'CC2019': 'NBE', 'ESTRATOS_P': 'No bosque', 'RASTERVALU': 2, 'ecorregion': 'amazonia', 'plotid': 702640}}, {'type': 'Feature', 'geometry': {'type': 'Point', 'coordinates': [-76.87476838837534, -2.1430707208924096]}, 'id': '00000000000000000092', 'properties': {'CC20132014': 'NBE', 'CC20152016': 'NBE', 'CC20172018': 'NBE', 'CC2019': 'NBE', 'ESTRATOS_P': 'No bosque', 'RASTERVALU': 2, 'ecorregion': 'am

In [3]:
# Grid Size (in degrees)
#grid_size = 0.1
# Generate the grid
grid, grid_fc = h.generate_grid(aoi, grid_size, grid_size, 0, 0)

# get a random point over each grid cell
randomInCell = grid_fc.map(h.get_random_point).filterBounds(aoi)

# get the centroid of each gridcell
centerInCell = grid_fc.map(h.get_center_point).filterBounds(aoi)

# stratified sampling based on ESA LC
imageCollection = ee.ImageCollection("ESA/WorldCover/v100")
esaLc = imageCollection.filterBounds(aoi).first().clip(aoi)

stratifiedSamples = esaLc.stratifiedSample(**{
    "numPoints": 100,
    "region": aoi,
    "scale": 100, 
    "seed": 42, 
    "tileScale": 4, 
    "geometries": True
}).map(h.set_id)

In [5]:
# intialize Map
Map = SepalMap(['HYBRID'])
Map.zoom_ee_object(aoi.geometry())

# add layers
Map.addLayer(ee.Image().byte().paint(featureCollection=aoi,color=1,width=3), {"palette": "blue"}, 'Area of interest')
Map.addLayer(esaLc, False, 'ESA Worldcover 2020')
Map.addLayer(stratifiedSamples.draw(color="black", pointRadius=2), {}, 'Stratified Samples per class from ESA LC')
Map.addLayer(ee.Image().byte().paint(featureCollection=grid,color=1,width=3), {"palette": "black"}, 'Grid')
Map.addLayer(randomInCell.draw(color="purple", pointRadius=2), {}, '1 Center Sample per gridcell (Systematic)')
Map.addLayer(centerInCell.draw(color="white", pointRadius=2), {}, '1 Random Sample per gridcell (Systematic random)')
#
Map

KeyboardInterrupt: 

### 3- Algorithm parameter settings

**Edit for advanced users, otherwise just execute**

In [4]:
# bfast parameters
bfast_params = {
    'run': bfast_monitor,
    'start_monitor': start_monitor, 
    'freq': 365,
    'k': 3, 
    'hfrac':0.25, 
    'trend': True, 
    'level':0.05, 
    'backend':'python'}

cusum_params = {
    'run': cusum_deforest,
    'nr_of_bootstraps': 1000
}

bs_slope_params = {
    'run': bs_slope,
    'nr_of_bootstraps': 1000
}

ts_metrics_params = {
    'run': ts_metrics,
    'outlier_removal': True,
    'z_threshhold': 3
}

ccdc_params = {
    'run': ccdc,
}

global_products = {
    'run': global_products,
}

### DO NOT CHANGE ###
### GATHER ALL INFO INTO A DICT #####
config_dict = {
    'work_dir': outdir,
    'workers': 12,
    'ts_params': {
        'start_date': start_date,
        'start_monitor': start_monitor,
        'end_date': end_date,
        'point_id': point_id_name,
        'grid_size': grid_size,
        'band': band,
        'satellite': satellite
    },
    'bfast_params': bfast_params,
    'cusum_params': cusum_params,
    'bs_slope_params': bs_slope_params,
    'ts_metrics_params': ts_metrics_params,
    'ccdc_params': ccdc_params,
    'global_products': global_products
}

# Run the whole thing

**Execute only**

In [ ]:
h.get_change_data(aoi, centerInCell, config_dict)

In [5]:
h.get_change_data(aoi, fc, config_dict)

 Parallelizing time-series extraction on 12 threads for a total of 49 grid cells.
 Processing gridcell 2
 Processing gridcell 1
 Processing gridcell 9
 Grid cell 0 does not contain any points. Going on with next grid cell.
 Grid cell 6 does not contain any points. Going on with next grid cell.
 Processing gridcell 11
 Processing gridcell 7
 Processing gridcell 10
 Processing gridcell 4
 Processing gridcell 8
 Processing gridcell 5
 Processing gridcell 3
 Processing gridcell 13
 Grid cell 12 does not contain any points. Going on with next grid cell.
 Processing gridcell 14


2022-07-04 16:36:30.277709: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-07-04 16:36:30.277757: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ca34987d3951): /proc/driver/nvidia/version does not exist
2022-07-04 16:36:30.372036: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


 Grid cell 1 with 6 points done in: 0:04:45.987728
 Processing gridcell 15
 Grid cell 13 with 9 points done in: 0:05:20.046754
 Processing gridcell 16
 Grid cell 3 with 4 points done in: 0:05:45.248008
 Grid cell 7 with 6 points done in: 0:05:46.133577
 Processing gridcell 17
 Grid cell 18 does not contain any points. Going on with next grid cell.
 Processing gridcell 19
 Grid cell 11 with 6 points done in: 0:06:13.464087
 Processing gridcell 20
 Grid cell 8 with 4 points done in: 0:07:47.608407
 Grid cell 2 with 4 points done in: 0:07:50.507696
 Processing gridcell 21
 Processing gridcell 22
 Grid cell 14 with 6 points done in: 0:08:02.020471
 Processing gridcell 23
 Grid cell 9 with 4 points done in: 0:08:53.867727
 Processing gridcell 24
 Grid cell 4 with 4 points done in: 0:09:40.809772
 Processing gridcell 25
 Grid cell 17 with 9 points done in: 0:06:42.303597
 Processing gridcell 26
 Grid cell 5 with 6 points done in: 0:13:15.748707
 Grid cell 10 with 4 points done in: 0:13:20.84

/usr/local/lib/python3.8/dist-packages/bfast/monitor/python/base.py:255: RuntimeWarning: invalid value encountered in sqrt
  sigma = np.sqrt(np.sum(y_error[:ns] ** 2) / (ns - (2 + 2 * self.k)))


 Grid cell 38 with 4 points done in: 0:19:51.928925
 Grid cell 44 with 4 points done in: 0:14:33.509151
 Grid cell 43 with 6 points done in: 0:15:33.737686
 Grid cell 45 with 4 points done in: 0:18:10.277248
 Grid cell 48 with 1 points done in: 0:17:32.035062
 Grid cell 46 with 4 points done in: 0:21:36.880811


/usr/local/lib/python3.8/dist-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,


 Processing has been finished successfully. Check for final_results files in your output directory.
